<a href="https://colab.research.google.com/github/AnshumanAI/Art-news-temp/blob/main/Art_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4
!pip install spacy
!pip install textblob
!pip install sklearn
!pip install feedparser
!python -m spacy download en_core_web_sm

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to scrape headlines from BBC News
def scrape_bbc_headlines(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')

        # Check the structure of the page and update the selector accordingly
        headlines = soup.find_all('div')  # Example selector
        if not headlines:
            print("No headlines found with the current selector.")

        headlines_text = [h3.get_text() for h3 in headlines if h3.get_text()]
        return headlines_text
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return []

# Example usage
bbc_headlines = scrape_bbc_headlines("https://www.bbc.com")
print(bbc_headlines[:10])  # Print the first 10 headlines for checking

['Skip to contentBritish Broadcasting CorporationWatchHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS ElectionUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC VerifySportBusinessExecutive LoungeTechnology of BusinessWomen at the HelmFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean & BermudaCentral AmericaEuropeMiddle EastNorth AmericaSouth AmericaWorld’s TableCulture & ExperiencesAdventuresThe SpeciaListEarthNatural WondersWeather & ScienceClimate SolutionsSustainable BusinessGreen LivingVideoLiveLive NewsLive SportHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveAudioWeatherNewslettersKatty Kay: Trump fights for spotlight as Democra

In [ ]:
import feedparser

# Example: BBC News RSS Feed
rss_url = "http://feeds.bbci.co.uk/news/rss.xml"
feed = feedparser.parse(rss_url)
news_headlines_rss = [entry.title for entry in feed.entries]

print(news_headlines_rss[:10])  # Print the first 10 headlines for checking

['How the tide turned after a week of riots', 'No refunds in historic miscarriage of justice cases', 'Police inbox for victims of violence not monitored', "Drug production booming in UK's empty high streets", 'What does science tell us about boxing’s gender row?', 'Vinted U-turns on delivery changes after backlash', 'Katie Price arrested for failing to attend court on return to UK', "Cash-strapped council pins hopes on 'family-style' children's home", 'Debate showdown between Trump and Harris set for September', 'Man dies competing in Texas CrossFit swimming event']


In [ ]:
import spacy
from collections import Counter

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

def extract_keywords(texts):
    keywords = []
    for text in texts:
        doc = nlp(text)
        keywords.extend([token.text for token in doc if token.is_alpha and not token.is_stop])
    return Counter(keywords).most_common(10)

keywords = extract_keywords(bbc_headlines)
print("Top Keywords:", keywords)

Top Keywords: [('hrs', 623), ('agoUS', 177), ('Trump', 150), ('days', 128), ('Canada', 93), ('state', 90), ('agoOlympic', 88), ('year', 85), ('day', 83), ('says', 79)]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Vectorize headlines using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(bbc_headlines)

# Cluster the headlines using K-Means
num_clusters = 5
km = KMeans(n_clusters=num_clusters, random_state=42)
km.fit(X)

# Assign each headline to a cluster
clusters = km.labels_.tolist()

# Print cluster assignments
for i, cluster in enumerate(clusters):
    print(f"Headline: {bbc_headlines[i]} - Cluster: {cluster}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Headline: Skip to contentBritish Broadcasting CorporationWatchHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS ElectionUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC VerifySportBusinessExecutive LoungeTechnology of BusinessWomen at the HelmFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean & BermudaCentral AmericaEuropeMiddle EastNorth AmericaSouth AmericaWorld’s TableCulture & ExperiencesAdventuresThe SpeciaListEarthNatural WondersWeather & ScienceClimate SolutionsSustainable BusinessGreen LivingVideoLiveLive NewsLive SportHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveAudioWeatherNewslettersKatty Kay: Trump fights for spotlight as

In [ ]:
from textblob import TextBlob

def analyze_sentiment(texts):
    sentiments = []
    for text in texts:
        blob = TextBlob(text)
        sentiments.append(blob.sentiment.polarity)
    return sentiments

sentiments = analyze_sentiment(bbc_headlines)
print("Sentiments:", sentiments)

Sentiments: [0.09941710923669683, 0.09941710923669683, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13636363636363635, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13636363636363635, 0.13636363636363635, 0.0, 0.0, 0.0, 0.0903409090909091, 0.0903409090909091, 0.18454545454545454, 0.16818181818181818, 0.2, 0.2, 0.2, 0.2, 0.0, 0.0, 0.0, 0.13636363636363635, 0.13636363636363635, 0.13636363636363635, 0.13636363636363635, 0.0, 0.0, 0.0, 0.16818181818181818, 0.2, 0.2, 0.2, 0.2, 0.0, 0.0, 0.0, 0.13636363636363635, 0.13636363636363635, 0.13636363636363635, 0.13636363636363635, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, -0.06666666666666667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.13636363636363635, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2604166666666667, 0.0, 0.0, 0.0, 0.3125, 0.12083333333333333, 0.12083

In [ ]:
# Example left/right keywords
left_keywords = ['progressive', 'liberal', 'climate change', 'equality']
right_keywords = ['conservative', 'tax cuts', 'freedom', 'immigration']

def detect_bias(headline):
    headline_lower = headline.lower()
    left_score = sum([headline_lower.count(word) for word in left_keywords])
    right_score = sum([headline_lower.count(word) for word in right_keywords])
    return 'left' if left_score > right_score else 'right' if right_score > left_score else 'neutral'

biases = [detect_bias(headline) for headline in bbc_headlines]
print("Biases:", biases)

Biases: ['right', 'right', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'right', 'right', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'right', 'neutral', 'neutral', 

In [ ]:
!pip freeze > requirements.txt

In [ ]:

import pandas as pd
df = pd.DataFrame(sentiments)
df.head(10)

,0
0,0.099417
1,0.099417
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
